In [4]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/ubuntu/varios/skforecast'

In [5]:
from skforecast.model_selection.model_selection import _create_backtesting_folds
from skforecast.model_selection import backtesting_forecaster
import pandas as pd
import numpy as np

In [6]:
y = pd.Series(np.arange(100))
y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
window_size = 3
initial_train_size = 60
test_size = 10
refit = False

folds = _create_backtesting_folds(
            data                  = y,
            window_size           = window_size,
            initial_train_size    = initial_train_size,
            test_size             = test_size,
            externally_fitted     = False,
            refit                 = 2,
            fixed_train_size      = True,
            gap                   = 0,
            skip_folds            = [2],
            allow_incomplete_fold = True,
            return_all_indexes    = False,
            verbose               = True
        )

folds

Information of backtesting process
----------------------------------
Number of observations used for initial training: 60
Number of observations used for backtesting: 40
    Number of folds: 4
    Number skipped folds: 1 [2]
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2022-01-01 00:00:00 -- 2022-03-01 00:00:00  (n=60)
    Validation: 2022-03-02 00:00:00 -- 2022-03-11 00:00:00  (n=10)
Fold: 1
    Training:   no training in this fold
    Validation: 2022-03-12 00:00:00 -- 2022-03-21 00:00:00  (n=10)
Fold: 2
    Fold skipped
Fold: 3
    Training:   no training in this fold
    Validation: 2022-04-01 00:00:00 -- 2022-04-10 00:00:00  (n=10)



[[[0, 60], [57, 60], [60, 70], [60, 70], False],
 [[0, 60], [67, 70], [70, 80], [70, 80], False],
 [[20, 80], [87, 90], [90, 100], [90, 100], False]]

In [7]:
y = pd.Series(np.arange(100))
y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
window_size = 3
initial_train_size = 60
test_size = 10
refit = False

folds = _create_backtesting_folds(
            data                  = y,
            window_size           = window_size,
            initial_train_size    = initial_train_size,
            test_size             = test_size,
            externally_fitted     = False,
            refit                 = 2,
            fixed_train_size      = True,
            gap                   = 0,
            skip_folds            = [2],
            allow_incomplete_fold = True,
            return_all_indexes    = False,
            verbose               = True
        )

folds

Information of backtesting process
----------------------------------
Number of observations used for initial training: 60
Number of observations used for backtesting: 40
    Number of folds: 4
    Number skipped folds: 1 [2]
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2022-01-01 00:00:00 -- 2022-03-01 00:00:00  (n=60)
    Validation: 2022-03-02 00:00:00 -- 2022-03-11 00:00:00  (n=10)
Fold: 1
    Training:   no training in this fold
    Validation: 2022-03-12 00:00:00 -- 2022-03-21 00:00:00  (n=10)
Fold: 2
    Fold skipped
Fold: 3
    Training:   no training in this fold
    Validation: 2022-04-01 00:00:00 -- 2022-04-10 00:00:00  (n=10)



[[[0, 60], [57, 60], [60, 70], [60, 70], False],
 [[0, 60], [67, 70], [70, 80], [70, 80], False],
 [[20, 80], [87, 90], [90, 100], [90, 100], False]]

In [9]:
# Libraries
# ==============================================================================
import pandas as pd
import matplotlib.pyplot as plt
from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "datetime"], "header": 0}
)

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data[['y']]
data = data.sort_index()

# Train-validation dates
# ==============================================================================
end_train = '2002-01-01 23:59:00'

print(
    f"Train dates      : {data.index.min()} --- {data.loc[:end_train].index.max()}"
    f"  (n={len(data.loc[:end_train])})"
)
print(
    f"Validation dates : {data.loc[end_train:].index.min()} --- {data.index.max()}"
    f"  (n={len(data.loc[end_train:])})"
)


# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 15 
             )

metric, predictions = backtesting_forecaster(
                          forecaster            = forecaster,
                          y                     = data['y'],
                          steps                 = 10,
                          metric                = 'mean_squared_error',
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = False,
                          gap                   = 0,
                          skip_folds            = 2,
                          allow_incomplete_fold = True,
                          refit                 = False,
                          n_jobs                = 'auto',
                          verbose               = True,
                          show_progress         = True  
                      )

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 2)
Train dates      : 1991-07-01 00:00:00 --- 2002-01-01 00:00:00  (n=127)
Validation dates : 2002-02-01 00:00:00 --- 2008-06-01 00:00:00  (n=77)
Information of backtesting process
----------------------------------
Number of observations used for initial training: 127
Number of observations used for backtesting: 77
    Number of folds: 8
    Number skipped folds: 4 [1, 3, 5, 7]
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 7 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 2002-01-01 00:00:00  (n=127)
    Validation: 2002-02-01 00:00:00 -- 2

  0%|          | 0/4 [00:00<?, ?it/s]